In [2]:
### Filter surface radiative calculations for sky-free conditions and 
### and combine into monthly files. 

In [4]:
import xarray as xr
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
import os
from glob import glob
from scipy.interpolate import interp1d
from scipy.ndimage.filters import gaussian_filter1d
from joblib import Parallel, delayed

from matplotlib.font_manager import FontProperties

from simulate_radiances_utils import *
# %run simulate_radiances_utils.py
import urllib 

HAPI version: 1.1.0.9.7
To get the most up-to-date version please check http://hitran.org/hapi
ATTENTION: Python versions of partition sums from TIPS-2017 are now available in HAPI code

           It is free to use HAPI. If you use HAPI in your research or software development,
           please cite it using the following reference:
           R.V. Kochanov, I.E. Gordon, L.S. Rothman, P. Wcislo, C. Hill, J.S. Wilzewski,
           HITRAN Application Programming Interface (HAPI): A comprehensive approach
           to working with spectroscopic data, J. Quant. Spectrosc. Radiat. Transfer 177, 15-30 (2016)
           DOI: 10.1016/j.jqsrt.2016.03.005


In [5]:
path_to_cloud_mask = '/net/fluo/data2/groupMembers/cchristo/misc_data/cloud_bool_rad_29.nc'

# rel_dir_to_all_surface_spec = '/net/fluo/data2/groupMembers/cchristo/results/rt_results/'
# output_rel_dir_to_all_surface_spec = '/net/fluo/data2/groupMembers/cchristo/results/rt_results_monthly/'

rel_dir_to_all_surface_spec = '/net/fluo/data2/groupMembers/cchristo/results/rt_results_preindus_CO2/'
output_rel_dir_to_all_surface_spec = '/net/fluo/data2/groupMembers/cchristo/results/rt_results_preindus_CO2_monthly/'


rel_dir_format = '{year}/{month:02d}/'
# fname_format = out_data_path_format = '{year}/{month:02d}/surface_fields_{year}{month:02d}{day:02d}_{hour:02d}{minute:02d}.nc'

In [8]:
all_months = pd.date_range(start = '2011-01',
                          end = '2016-01',
                          freq = 'M')

In [14]:
all_months[-1]

Timestamp('2015-12-31 00:00:00', freq='M')

In [6]:
cloud_mask_ds = xr.open_dataset(path_to_cloud_mask)

In [ ]:
def process_time_parallel(time_i):
    print(time_i)
    out_fpath = output_rel_dir_to_all_surface_spec + 'surface_fields_{year}{month:02d}.nc'.format(year = time_i.year,
                                                         month = time_i.month)
    if not os.path.exists(out_fpath):
        rel_dir = rel_dir_format.format(year = time_i.year, month = time_i.month)
        file_names = os.listdir(rel_dir_to_all_surface_spec + rel_dir)
        file_paths_i = [rel_dir_to_all_surface_spec + rel_dir + filename for filename in file_names]

        # get cloud-free mask for month
        cloud_mask_ds_i = cloud_mask_ds.sel(time = '{year}-{month:02d}'.format(year = time_i.year, 
                                                        month = time_i.month))
        # find only cloud-free times 
        cloud_free_ds = cloud_mask_ds_i.where(cloud_mask_ds_i['cloud_free_bool'] == 1, drop = True)

        # open surface spectrums for month 
        ds_i = xr.open_mfdataset(file_paths_i, combine='by_coords')

        # select and save only cloud_free times
        ds_i_cloud_free = ds_i.sel(time = cloud_free_ds.time)

        out_fpath = output_rel_dir_to_all_surface_spec + 'surface_fields_{year}{month:02d}.nc'.format(year = time_i.year,
                                                             month = time_i.month)

        ds_i_cloud_free.to_netcdf(out_fpath)

In [ ]:
Parallel(n_jobs=20)(delayed(process_time_parallel)(time_i) for time_i in all_months)
print('DONE!')

# Not parallel

In [7]:
# for time_i in all_months:
#     print(time_i)
#     out_fpath = output_rel_dir_to_all_surface_spec + 'surface_fields_{year}{month:02d}.nc'.format(year = time_i.year,
#                                                          month = time_i.month)
#     if not os.path.exists(out_fpath):
#         rel_dir = rel_dir_format.format(year = time_i.year, month = time_i.month)
#         file_names = os.listdir(rel_dir_to_all_surface_spec + rel_dir)
#         file_paths_i = [rel_dir_to_all_surface_spec + rel_dir + filename for filename in file_names]

#         # get cloud-free mask for month
#         cloud_mask_ds_i = cloud_mask_ds.sel(time = '{year}-{month:02d}'.format(year = time_i.year, 
#                                                         month = time_i.month))
#         # find only cloud-free times 
#         cloud_free_ds = cloud_mask_ds_i.where(cloud_mask_ds_i['cloud_free_bool'] == 1, drop = True)

#         # open surface spectrums for month 
#         ds_i = xr.open_mfdataset(file_paths_i, combine='by_coords')

#         # select and save only cloud_free times
#         ds_i_cloud_free = ds_i.sel(time = cloud_free_ds.time)

#         out_fpath = output_rel_dir_to_all_surface_spec + 'surface_fields_{year}{month:02d}.nc'.format(year = time_i.year,
#                                                              month = time_i.month)

#         ds_i_cloud_free.to_netcdf(out_fpath)

2011-01-31 00:00:00
2011-02-28 00:00:00
2011-03-31 00:00:00
2011-04-30 00:00:00
2011-05-31 00:00:00
2011-06-30 00:00:00
2011-07-31 00:00:00
2011-08-31 00:00:00
2011-09-30 00:00:00
2011-10-31 00:00:00
2011-11-30 00:00:00
2011-12-31 00:00:00
2012-01-31 00:00:00
2012-02-29 00:00:00
2012-03-31 00:00:00
2012-04-30 00:00:00
2012-05-31 00:00:00
2012-06-30 00:00:00
2012-07-31 00:00:00
2012-08-31 00:00:00
2012-09-30 00:00:00
2012-10-31 00:00:00
2012-11-30 00:00:00
2012-12-31 00:00:00
2013-01-31 00:00:00


KeyboardInterrupt: 

<xarray.DataArray 'time' (time: 113)>
array(['2011-02-01T01:30:00.000000000', '2011-02-02T10:30:00.000000000',
       '2011-02-02T13:30:00.000000000', '2011-02-02T16:30:00.000000000',
       '2011-02-02T22:30:00.000000000', '2011-02-03T01:30:00.000000000',
       '2011-02-03T04:30:00.000000000', '2011-02-03T13:30:00.000000000',
       '2011-02-03T16:30:00.000000000', '2011-02-03T19:30:00.000000000',
       '2011-02-03T22:30:00.000000000', '2011-02-04T01:30:00.000000000',
       '2011-02-04T04:30:00.000000000', '2011-02-04T07:30:00.000000000',
       '2011-02-04T10:30:00.000000000', '2011-02-04T16:30:00.000000000',
       '2011-02-04T19:30:00.000000000', '2011-02-04T22:30:00.000000000',
       '2011-02-05T01:30:00.000000000', '2011-02-05T04:30:00.000000000',
       '2011-02-05T10:30:00.000000000', '2011-02-05T13:30:00.000000000',
       '2011-02-05T16:30:00.000000000', '2011-02-05T19:30:00.000000000',
       '2011-02-05T22:30:00.000000000', '2011-02-06T01:30:00.000000000',
       '2011-02-06T04:30:00.000000000', '2011-02-06T07:30:00.000000000',
       '2011-02-06T10:30:00.000000000', '2011-02-06T13:30:00.000000000',
       '2011-02-06T16:30:00.000000000', '2011-02-06T19:30:00.000000000',
       '2011-02-06T22:30:00.000000000', '2011-02-07T01:30:00.000000000',
       '2011-02-07T04:30:00.000000000', '2011-02-07T07:30:00.000000000',
       '2011-02-07T10:30:00.000000000', '2011-02-07T13:30:00.000000000',
       '2011-02-07T22:30:00.000000000', '2011-02-08T01:30:00.000000000',
       '2011-02-08T04:30:00.000000000', '2011-02-08T07:30:00.000000000',
       '2011-02-08T10:30:00.000000000', '2011-02-08T22:30:00.000000000',
       '2011-02-09T10:30:00.000000000', '2011-02-09T19:30:00.000000000',
       '2011-02-09T22:30:00.000000000', '2011-02-10T01:30:00.000000000',
       '2011-02-10T04:30:00.000000000', '2011-02-10T13:30:00.000000000',
       '2011-02-10T16:30:00.000000000', '2011-02-10T19:30:00.000000000',
       '2011-02-11T01:30:00.000000000', '2011-02-12T16:30:00.000000000',
       '2011-02-12T19:30:00.000000000', '2011-02-15T04:30:00.000000000',
       '2011-02-15T07:30:00.000000000', '2011-02-15T13:30:00.000000000',
       '2011-02-15T19:30:00.000000000', '2011-02-15T22:30:00.000000000',
       '2011-02-16T01:30:00.000000000', '2011-02-16T04:30:00.000000000',
       '2011-02-16T13:30:00.000000000', '2011-02-16T16:30:00.000000000',
       '2011-02-16T19:30:00.000000000', '2011-02-16T22:30:00.000000000',
       '2011-02-17T01:30:00.000000000', '2011-02-17T04:30:00.000000000',
       '2011-02-17T07:30:00.000000000', '2011-02-17T10:30:00.000000000',
       '2011-02-17T13:30:00.000000000', '2011-02-17T16:30:00.000000000',
       '2011-02-17T19:30:00.000000000', '2011-02-18T01:30:00.000000000',
       '2011-02-18T10:30:00.000000000', '2011-02-19T13:30:00.000000000',
       '2011-02-19T19:30:00.000000000', '2011-02-19T22:30:00.000000000',
       '2011-02-20T04:30:00.000000000', '2011-02-20T10:30:00.000000000',
       '2011-02-20T13:30:00.000000000', '2011-02-20T16:30:00.000000000',
       '2011-02-20T19:30:00.000000000', '2011-02-20T22:30:00.000000000',
       '2011-02-21T01:30:00.000000000', '2011-02-21T04:30:00.000000000',
       '2011-02-21T07:30:00.000000000', '2011-02-21T10:30:00.000000000',
       '2011-02-22T01:30:00.000000000', '2011-02-22T10:30:00.000000000',
       '2011-02-22T16:30:00.000000000', '2011-02-22T19:30:00.000000000',
       '2011-02-22T22:30:00.000000000', '2011-02-23T01:30:00.000000000',
       '2011-02-23T07:30:00.000000000', '2011-02-23T10:30:00.000000000',
       '2011-02-25T04:30:00.000000000', '2011-02-25T07:30:00.000000000',
       '2011-02-25T13:30:00.000000000', '2011-02-25T16:30:00.000000000',
       '2011-02-25T22:30:00.000000000', '2011-02-26T01:30:00.000000000',
       '2011-02-26T04:30:00.000000000', '2011-02-26T13:30:00.000000000',
       '2011-02-26T22:30:00.000000000', '2011-02-27T04:30:00.000000000',
       '2011-02-27T13:30:00.000000000', '2011-02-27T16:30:00.000000000',
       '2011-02-27T1

In [75]:
# cloud_free_ds


In [5]:
# cloud_mask_ds

<xarray.Dataset>
Dimensions:          (time: 14609)
Coordinates:
  * time             (time) datetime64[ns] 2011-01-01T01:30:00 ... 2016-01-01T01:30:00
Data variables:
    cloud_free_bool  (time) float64 ...
    800_cm_mean_rad  (time) float64 ...

In [27]:
# plt.plot(cloud_mask_ds['cloud_free_bool'].values[250:500])

In [28]:
# plt.plot(cloud_mask_ds['800_cm_mean_rad'].values[250:500])